<a href="https://colab.research.google.com/github/superahxd/_project/blob/main/sec4/twint_scrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install twint
!pip install nest_asyncio
!pip3 install --user --upgrade git+https://github.com/twintproject/twint.git@origin/master#egg=twint

  Cloning https://github.com/twintproject/twint.git (to revision origin/master) to /tmp/pip-install-3zptbwh3/twint_7e04a8ecf6cd4e8a9addce562fc756d2
  Running command git clone -q https://github.com/twintproject/twint.git /tmp/pip-install-3zptbwh3/twint_7e04a8ecf6cd4e8a9addce562fc756d2
  Running command git checkout -q origin/master


In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import twint

c = twint.Config()

c.Limit = 200000
c.Search = 'DokeV'
c.Since = '2021-08-20'
c.Until = '2021-09-01'
c.Output = 'practice3.json'
c.Popular_tweets = True
c.Store_json = True
c.Hide_output = True

twint.run.Search(c)

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.


In [ ]:
import pandas as pd

df = pd.read_json('practice3.json', lines=True)
df[["created_at", "id", "tweet", "retweets_count"]]

,created_at,id,tweet,retweets_count
0,2021-08-31 23:42:33+00:00,1432851332763897867,@DokeVGame Are the platform releases announced?,0
1,2021-08-31 23:35:38+00:00,1432849590215065600,한국경관을 담은 메타버스 게임 도깨비(Dokev) 공개···가상경관 설계시 중요한 ...,0
2,2021-08-31 23:28:22+00:00,1432847762282921984,@DokeVGame How you ganna show me this game and...,0
3,2021-08-31 23:02:23+00:00,1432841223308464133,All hail DokeV #DokeV #Pokémon #TemTem #Ninten...,1
4,2021-08-31 22:59:01+00:00,1432840379150200837,あ、配信始めたらの話なんだけどリリースもうちょっと先だけど、DokeV（ドケビ）って言うmm...,0
...,...,...,...,...
6203,2021-08-26 02:23:14+00:00,1430717443521929220,DokeV recebe trailer em tempo real onde muita ...,0
6204,2021-08-26 02:23:08+00:00,1430717418175811586,"@DokeVGame So cute❤ I love this, I can't wait😍😍",0
6205,2021-08-26 02:22:22+00:00,1430717225288208391,I guess I need to spread the word too- DokeV i...,0
6206,2021-08-26 02:21:57+00:00,1430717121399455746,@geoffkeighley DokeV,0


In [ ]:
# utilities
import re
import pickle
import numpy as np
import pandas as pd

# plotting
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# nltk
from nltk.stem import WordNetLemmatizer

# sklearn
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
dataset = df[['tweet']]

In [ ]:
text = list(dataset['tweet'])

In [ ]:
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import nltk

nltk.download('stopwords')

stopwordlist = stopwords.words('english')
stemmer = SnowballStemmer('english')
emojis = {':)': 'smile', ':-)': 'smile', ';d': 'wink', ':-E': 'vampire', ':(': 'sad', 
          ':-(': 'sad', ':-<': 'sad', ':P': 'raspberry', ':O': 'surprised',
          ':-@': 'shocked', ':@': 'shocked',':-$': 'confused', ':\\': 'annoyed', 
          ':#': 'mute', ':X': 'mute', ':^)': 'smile', ':-&': 'confused', '$_$': 'greedy',
          '@@': 'eyeroll', ':-!': 'confused', ':-D': 'smile', ':-0': 'yell', 'O.o': 'confused',
          '<(-_-)>': 'robot', 'd[-_-]b': 'dj', ":'-)": 'sadsmile', ';)': 'wink', 
          ';-)': 'wink', 'O:-)': 'angel','O*-)': 'angel','(:-D': 'gossip', '=^.^=': 'cat'}

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def preprocess(textdata):
    processedText = []
    
    # Create Lemmatizer and Stemmer. 표제어 추출하기
    wordLemm = WordNetLemmatizer()
    
    # Defining regex patterns. 정규식 패턴
    urlPattern        = r"((http://)[^ ]*|(https://)[^ ]*|( www\.)[^ ]*)" 
    userPattern       = '@[^\s]+' #@으로 시작하는 유저 아이디 제거
    alphaPattern      = "[^a-zA-Z]" #영어 대소문자 외 제거
    sequencePattern   = r"(.)\1\1+" #중복글자 제거
    seqReplacePattern = r"\1\1" #중복글자제거
    
    for tweet in textdata:
        tweet = tweet.lower() #소문자화
        #http, www, https로 시작하는 url 제거
        tweet = re.sub(urlPattern,' URL',tweet)
        # 이모지 대체
        for emoji in emojis.keys():
            tweet = tweet.replace(emoji, "EMOJI" + emojis[emoji])        
        # 유저 아이디 대체
        tweet = re.sub(userPattern,' USER', tweet)        
        # 알파벳 아닌 것 모두 대체
        tweet = re.sub(alphaPattern, " ", tweet)
        # 3개 이상의 연속된 문자를 2자로 대체
        tweet = re.sub(sequencePattern, seqReplacePattern, tweet)

        tweetwords = ''
        for word in tweet.split():
            # stopword에 들어있는지 확인
            # stopword에 없으면:
            if len(word)>1:
                # Lemmatizing 실행
                word = wordLemm.lemmatize(word)
                tweetwords += (word+' ')
            
        processedText.append(tweetwords)
        
    return processedText

In [ ]:
import time

nltk.download('wordnet')

t = time.time()
processedtext = preprocess(text)
print(f'Text Preprocessing complete.')
print(f'Time Taken: {round(time.time()-t)} seconds')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Text Preprocessing complete.
Time Taken: 2 seconds


In [ ]:
df = pd.DataFrame(processedtext)

In [ ]:
df.to_csv('/content/last.csv')